# Comment écrire un nombre à virgule ?
## L'encodage des nombres, un choix numérique

Nous avons appris à encoder des nombres entiers naturels et relatifs, et nous avons vu que __les limites physiques des machines imposent des limites sur l'étendue des valeurs__. 

Par exemple, sur un octet on dispose de 2$^8$ = 256 valeurs distinctes qui permettent d'encoder, au choix :

- des nombres entiers naturels entre 0 et 255.
- des nombres entiers relatifs entre -128 et 127.

Maintenant que nous allons tenter de __coder les réels__, les limites de notre machine vont encore entraîner des __limites__ sur l'étendue des valeurs, mais également sur la __précision des valeurs__.

Malgré sont apparente rigueur, les exemples suivants devraient vous convaincre que l'encodage utilisé par Python présente des failles...

In [ ]:
0.2 + 0.1 == 0.3

In [ ]:
0.2 + 0.1

Maintenant qu'il est acté qu'un encodage parfait dans ce domaine n'a pas été trouvé pour Python, voyons les alternatives qui s'ouvraient pour coder un nombre à virgule.

## Virgule fixe ou virgule flottante ?

Il existe deux façons de coder les nombres à virgule.

### Codage en virgule fixe

Le codage en virgule fixe consiste à __fixer la position de la virgule__. 

Par exemple, __sur un octet__, on peut décider de consacrer :

- les 4 premiers bits pour la partie entière du nombre.
- les 4 derniers bits pour la partie décimale.

> __Exemple :__ (1010, 1100)$_2$ = 2$³$ + 2$¹$ + 2$^{−1}$ + 2$^{−2}$ = (8 + 2),(0,5 + 0,25) = (10,75)$_{10}$

#### Application

1. Convertir `111, 001101` en notation décimale.

2. Convertir 0,54 en notation binaire à virgule fixe, avec la partie fractionnaire exprimée sur 4 bits. On présentera un résultat arrondi, au plus près.

### Codage en virgule flottante

Dans le codage __en virgule flottante, on privilégie un nombre fixe de chiffres significatifs__ et on réserve l'autre partie du codage à l'exposant. C'est une méthode __semblable à la notation scientifique__ utilisée en physique par exemple. Mais, au lieu de se baser sur des puissances de 10, on utilisera des __puissances de 2__ :

> ± m×2$^e$
- avec m ∈ $[1, 2[$
- m sera appelée __la mantisse__
- $n$ sera appelé __l'exposant__

C'est ce codage qui sera souvent privilégié dans les langages de programmation et, en particulier, avec Python : __les nombres flottants, de type `float`__.

## Principe du codage en virgule flottante

On représente un nombre à virgule flottante sous la forme : __(−1)$^s$×m×2$^e$__

[![By GMjeanmatt - Own work, CC BY-SA 3.0](IEEE754_Format_General.png)](https://commons.wikimedia.org/w/index.php?curid=7318209)

Supposons un nombre flottant codé sur __un octet__ utilisant __1 bit de signe, 3 bits pour l'exposant et 4 bits pour la mantisse__ : `1 101 1011`

- __s est le signe__, représenté par le bit de poids fort:
  - s=0 : signe +
  - s=1 : signe −
  
Notre exemple de codage représente donc un nombre négatif.

- __e est l'exposant, un entier compris entre e$_{min}$ et e$_{max}$ représenté par un entier relatif décalé__ et non en complément à deux, afin de faciliter la comparaison des exposants. Ce __décalage est de 2$^{n-1}$ - 1__ (n représente le nombre de bits utilisé pour coder l'exposant).

Dans notre exemple, l'exposant a pour valeur `101`, codé sur 3 bits, il doit être décalé de 2$^{3-1}$ - 1 = 3.  
Ainsi, puisque (101)$_2$ = (5)$_{10}$, l'exposant `101` correspond à un exposant de 5-3 = 2.

- __m est la mantisse qui est un nombre binaire à virgule compris entre 1 inclus et 2 exclus__. Le seul __chiffre avant la virgule étant toujours 1, il n'est pas représenté__ (on le dit "implicite"), et le codage binaire de la mantisse représente donc uniquement les chiffres après la virgule. Ces chiffres représentent un nombre codé en base 2 : __la somme des demis (2$^{−1}$), des quarts (2$^{−2}$), des huitièmes (2$^{−3}$),...__

Dans notre exemple, la mantisse est : `1011`, elle représentera le nombre : 2$^{−1}$ + 2$^{−3}$ + 2$^{−4}$ = 0,5 + 0,125 + 0,0625 = 0,6875.

__Le code `1 101 1011` sur un octet__, utilisant 1 bit de signe, 3 bits pour l'exposant et 4 bits pour la mantisse représente donc : −1,6875×2$^2$ = __−6,75__

## La norme IEEE 754

> L’IEEE 754 est une norme pour la représentation des nombres à virgule flottante en binaire. Elle est la __norme la plus employée actuellement__ pour le calcul des nombres à virgule flottante dans le domaine informatique. [Source Wikipedia](https://fr.wikipedia.org/wiki/IEEE_754)

Cette norme définit notamment __2 formats__ pour représenter des nombres à virgule flottante.

- __simple précision__ (__32 bits__ : 1 bit de signe, 8 bits d'exposant (-126 à 127), 23 bits de mantisse).

[![By GMjeanmatt - Own work, CC BY-SA 3.0](IEEE754_simple_precision.png)](https://commons.wikimedia.org/w/index.php?curid=7318385)

- __double précision (utilisé par défaut par Python)__ (__64 bits__ : 1 bit de signe, 11 bits d'exposant (-1022 à 1023), 52 bits de mantisse).
  
[![By GMjeanmatt - Own work, CC BY-SA 3.0](IEEE754_double_precision.png)](https://commons.wikimedia.org/w/index.php?curid=7318466)

#### Application (Approfondissement)

1. __Calculer__ le nombre représenté en __double précision__ par `1 10110100000 011101100000 ... 0`.

2. Réaliser la __conversion en flottant simple précision__ du nombre -10,125.
  
## Retour sur les tests d'égalité sur les flottants

Comme nous sommes limités sur la précision des flottants, __on transforme les tests d'égalités en tests d'inégalités, à une précision donnée__.

Si on retente le test initial, plus de surprise...

In [ ]:
0.2 + 0.1 == 0.3

Par contre, en utilisant une précision du millionième :

In [ ]:
precision = 1e-6
abs((0.2 + 0.1) - 0.3)  < precision

D'ailleurs la bibliothèque `math` de Python inclut à cet effet la [fonction `math.isclose()`](https://docs.python.org/3/library/math.html#math.isclose).

In [ ]:
from math import isclose
isclose(0.2 + 0.1, 0.3)

## Que retenir ?
### À minima...

- Python utilise un codage à virgule flottante pour représenter les nombres à virgule.
- La méthode à virgule flottante représente un nombre en écriture scientifique, en utlisant des puissances de 2 à la place des puissances de 10.
- La méthode à virgule flottante privilégie la précision à l'exactitude.
- Le codage des nombres flottants étant rarement exact, il faut éviter de les comparer en cherchant à savoir s'ils sont égaux entre eux.

### Au mieux...

- On peut représenter un nombre à virgule flottante de la manière suivante : (−1)$^s$×m×2$^e$
  - s est le signe
  - m est la mantisse
  - e est l'exposant
- Python utilise par défaut 64 bits pour coder un nombre flottant (double précision).
- Python utilise :
  - le bit de poids fort pour coder le signe.
  - quelques bits pour coder l'exposant (qui détermine l'ordre de grandeur du nombre).
  - la majorité des bits pour coder la mantisse (qui détermine le nombre de chiffres significatifs du nombre, donc sa précision).
- L’IEEE 754 est une norme pour la représentation des nombres à virgule flottante en binaire.
- Si on veut savoir si deux nombres flottants sont "égaux", on cherche à le déterminer avec un certain niveau d'imprécision, avec la fonction `isclose()` par exemple.
- Savoir représenter des nombres à virgules en nombres flottants à 8 bits.
- Savoir déterminer la valeur décimale de nombres flottants à 8 bits.

---
[![Licence CC BY NC SA](https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png "licence Creative Commons CC BY-NC-SA")](http://creativecommons.org/licenses/by-nc-sa/3.0/fr/)

<p style="text-align: center;">David Landry, Lycée Clemenceau - Nantes</p>

<p style="text-align: center;">D'après des cours partagés par...</p>
<p style="text-align: center;"><a  href=https://pixees.fr/informatiquelycee/n_site/nsi_prem.html>David Roche</a> sous la licence CC BY-SA</p>
<p style="text-align: center;"><a  href=https://www.lyceum.fr/1g/nsi>Benjamin Abel</a> sous la licence CC BY-SA</p>